In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk import corpus, tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer,porter

nltk.download('punkt')
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\achyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\achyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
df = pd.read_csv("D:\Learnbay\ML Real Time Project\Sentiment Analysis\hotstar_reviews.csv")
df.head()

,ID,UserName,Created_Date,Reviews,Lower_Case_Reviews,Sentiment_Manual_BP,Sentiment_Manual,Review_Length,DataSource,Year,Month,Date,Sentiment_Polarity
0,1,NaN,08-10-2017,Hh,hh,Negative,Negative,2,Google_PlayStore,2017,8,10,Neutral
1,2,NaN,08-11-2017,No,no,Negative,Negative,2,Google_PlayStore,2017,8,11,Neutral
2,3,asadynwa,08-12-2017,@hotstar_helps during paymnt for premium subsc...,@hotstar_helps during paymnt for premium subsc...,Help,Negative,140,Twitter,2017,8,12,Negative
3,4,jineshroxx,08-11-2017,@hotstartweets I am currently on Jio network a...,@hotstartweets i am currently on jio network a...,Help,Negative,140,Twitter,2017,8,11,Positive
4,5,YaminiSachar,08-05-2017,@hotstartweets the episodes of Sarabhai vs Sar...,@hotstartweets the episodes of sarabhai vs sar...,Help,Negative,140,Twitter,2017,8,5,Neutral


In [3]:
df1 = df[['Reviews','Sentiment_Polarity']]
df1.head()

,Reviews,Sentiment_Polarity
0,Hh,Neutral
1,No,Neutral
2,@hotstar_helps during paymnt for premium subsc...,Negative
3,@hotstartweets I am currently on Jio network a...,Positive
4,@hotstartweets the episodes of Sarabhai vs Sar...,Neutral


In [4]:
df2 = df1.copy()
df2

,Reviews,Sentiment_Polarity
0,Hh,Neutral
1,No,Neutral
2,@hotstar_helps during paymnt for premium subsc...,Negative
3,@hotstartweets I am currently on Jio network a...,Positive
4,@hotstartweets the episodes of Sarabhai vs Sar...,Neutral
...,...,...
5048,I LOATHE this application in so many levels Fo...,Negative
5049,1 Cannot keep track of progress If you watch v...,Neutral
5050,1 Miles below experience like Amazon prime 2 E...,Neutral
5051,You pay for premium your internet speed is at ...,Positive


In [5]:
# checking whether it works or not
a = "india is very beautiful place to visit"
sia.polarity_scores(a)

{'neg': 0.0, 'neu': 0.589, 'pos': 0.411, 'compound': 0.6361}

In [6]:
df1['Sentiment_Polarity'].value_counts()

Positive    2513
Neutral     1579
Negative     961
Name: Sentiment_Polarity, dtype: int64

In [7]:
df1.loc[5]['Reviews']

'@hotstartweets Not able 2 watch the latest episode of GoT on the app doesn t allow to take the screenshot of the error help to resolve ASAP'

In [8]:
sia.polarity_scores(df1.loc[5]['Reviews'])

{'neg': 0.087, 'neu': 0.68, 'pos': 0.233, 'compound': 0.5423}

In [9]:
df1.isnull().sum()

Reviews               0
Sentiment_Polarity    0
dtype: int64

In [10]:
# Preprocessing
  # missing values - no
  # encoding- will handle by SentimentIntensityAnalyzer
  # outlier - not required as it is text
  # scaling - not required as it is text
  # balance - data is already balanced

In [11]:
df1['Score'] = df1['Reviews'].apply(lambda rev:sia.polarity_scores(rev))
df1.head(10)

C:\Users\achyu\AppData\Local\Temp\ipykernel_32664\2497865032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Score'] = df1['Reviews'].apply(lambda rev:sia.polarity_scores(rev))


,Reviews,Sentiment_Polarity,Score
0,Hh,Neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,No,Neutral,"{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
2,@hotstar_helps during paymnt for premium subsc...,Negative,"{'neg': 0.102, 'neu': 0.898, 'pos': 0.0, 'comp..."
3,@hotstartweets I am currently on Jio network a...,Positive,"{'neg': 0.0, 'neu': 0.902, 'pos': 0.098, 'comp..."
4,@hotstartweets the episodes of Sarabhai vs Sar...,Neutral,"{'neg': 0.063, 'neu': 0.84, 'pos': 0.097, 'com..."
5,@hotstartweets Not able 2 watch the latest epi...,Neutral,"{'neg': 0.087, 'neu': 0.68, 'pos': 0.233, 'com..."
6,Please allow RuPay or Maestro payment gateways...,Positive,"{'neg': 0.0, 'neu': 0.673, 'pos': 0.327, 'comp..."
7,@hotstar_helps why today s epi of #LoveKaHaiIn...,Positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
8,@hotstartweets #HotStarFraud I paid for the su...,Positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
9,@hotstartweets i have a premium accnt at hotst...,Neutral,"{'neg': 0.0, 'neu': 0.929, 'pos': 0.071, 'comp..."


In [12]:
df1['compound'] = df1['Score'].apply(lambda score_dict:score_dict['compound'])


C:\Users\achyu\AppData\Local\Temp\ipykernel_32664\3957315183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['compound'] = df1['Score'].apply(lambda score_dict:score_dict['compound'])


In [13]:
df1.head(10)

,Reviews,Sentiment_Polarity,Score,compound
0,Hh,Neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
1,No,Neutral,"{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...",-0.2960
2,@hotstar_helps during paymnt for premium subsc...,Negative,"{'neg': 0.102, 'neu': 0.898, 'pos': 0.0, 'comp...",-0.2846
3,@hotstartweets I am currently on Jio network a...,Positive,"{'neg': 0.0, 'neu': 0.902, 'pos': 0.098, 'comp...",0.3612
4,@hotstartweets the episodes of Sarabhai vs Sar...,Neutral,"{'neg': 0.063, 'neu': 0.84, 'pos': 0.097, 'com...",0.2023
5,@hotstartweets Not able 2 watch the latest epi...,Neutral,"{'neg': 0.087, 'neu': 0.68, 'pos': 0.233, 'com...",0.5423
6,Please allow RuPay or Maestro payment gateways...,Positive,"{'neg': 0.0, 'neu': 0.673, 'pos': 0.327, 'comp...",0.8294
7,@hotstar_helps why today s epi of #LoveKaHaiIn...,Positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
8,@hotstartweets #HotStarFraud I paid for the su...,Positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
9,@hotstartweets i have a premium accnt at hotst...,Neutral,"{'neg': 0.0, 'neu': 0.929, 'pos': 0.071, 'comp...",0.1154


In [14]:
df1['compound_score'] = df1['compound'].apply(lambda c:'Positive' if c>=0.1 else('Negative' if c<=-0.1 else 'Neutral'))


C:\Users\achyu\AppData\Local\Temp\ipykernel_32664\656332761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['compound_score'] = df1['compound'].apply(lambda c:'Positive' if c>=0.1 else('Negative' if c<=-0.1 else 'Neutral'))


In [15]:
# Another way of writing code
"""
def comp_score(compound):
    if compound >=0.1:
        return 'pos'
    elif compound <=-0.1:
        return 'neg'
    else:
        return 'neu'
    
df1['compound_score2'] = df1['compound'].apply(comp_score)    
    """

"\ndef comp_score(compound):\n    if compound >=0.1:\n        return 'pos'\n    elif compound <=-0.1:\n        return 'neg'\n    else:\n        return 'neu'\n    \ndf1['compound_score2'] = df1['compound'].apply(comp_score)    \n    "

In [16]:
df1

,Reviews,Sentiment_Polarity,Score,compound,compound_score
0,Hh,Neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral
1,No,Neutral,"{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...",-0.2960,Negative
2,@hotstar_helps during paymnt for premium subsc...,Negative,"{'neg': 0.102, 'neu': 0.898, 'pos': 0.0, 'comp...",-0.2846,Negative
3,@hotstartweets I am currently on Jio network a...,Positive,"{'neg': 0.0, 'neu': 0.902, 'pos': 0.098, 'comp...",0.3612,Positive
4,@hotstartweets the episodes of Sarabhai vs Sar...,Neutral,"{'neg': 0.063, 'neu': 0.84, 'pos': 0.097, 'com...",0.2023,Positive
...,...,...,...,...,...
5048,I LOATHE this application in so many levels Fo...,Negative,"{'neg': 0.097, 'neu': 0.743, 'pos': 0.16, 'com...",0.8283,Positive
5049,1 Cannot keep track of progress If you watch v...,Neutral,"{'neg': 0.152, 'neu': 0.755, 'pos': 0.093, 'co...",-0.6478,Negative
5050,1 Miles below experience like Amazon prime 2 E...,Neutral,"{'neg': 0.049, 'neu': 0.878, 'pos': 0.073, 'co...",0.0900,Neutral
5051,You pay for premium your internet speed is at ...,Positive,"{'neg': 0.174, 'neu': 0.712, 'pos': 0.113, 'co...",-0.8523,Negative


## Evaluation Matrix

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

In [18]:
print(confusion_matrix(df1['Sentiment_Polarity'],df1['compound_score']))
print("**************"*6)
print(classification_report(df1['Sentiment_Polarity'],df1['compound_score']))
print("**************"*6)
print(accuracy_score(df1['Sentiment_Polarity'],df1['compound_score']))



[[ 566  285  110]
 [ 259 1043  277]
 [  86  503 1924]]
************************************************************************************
              precision    recall  f1-score   support

    Negative       0.62      0.59      0.60       961
     Neutral       0.57      0.66      0.61      1579
    Positive       0.83      0.77      0.80      2513

    accuracy                           0.70      5053
   macro avg       0.67      0.67      0.67      5053
weighted avg       0.71      0.70      0.70      5053

************************************************************************************
0.6991886008311894


In [19]:
# As the accuracy is less, lets try with ML approach

## Appraoch-2:ML

In [21]:
df2.head()

,Reviews,Sentiment_Polarity
0,Hh,Neutral
1,No,Neutral
2,@hotstar_helps during paymnt for premium subsc...,Negative
3,@hotstartweets I am currently on Jio network a...,Positive
4,@hotstartweets the episodes of Sarabhai vs Sar...,Neutral


In [25]:
df2['Reviews'].values

array(['Hh', 'No',
       '@hotstar_helps during paymnt for premium subscription the transaction failed twice but i have not received refund for one of the transaction',
       ...,
       '1 Miles below experience like Amazon prime 2 Even premium subscription trial has ads 3 Most of the videos take long to lead because of application seeking server for ads everytime you open a video or doing some nonsense checks 4 Most of the premium content cannot be downloaded and watched later And some content doesn t have subtitles when cast 5 You have to play a video to download it because the download option is not before but after you start playing I mean really 6 All the above may force me to stop subscription post 3 months or less',
       'You pay for premium your internet speed is at 20mbps and your Hotstar app either fails to load or tests your patience in such a way you d beat The Buddha at attaining Nirvana Blaming technology is an easy excuse Maintaining service standards for customers 

In [45]:
"""
from tqdm import tqdm
 
preprocessed_reviews = []
for sentence in tqdm(df2['Reviews'].values):
    sentence = re.sub(r'[^a-zA-Z]','',sentence)
    sentence = sentence.lower()
    sentence = sentence.split()
    sentence = [word for word in sentence if word not in set(stopwords.words('english'))]
    sentence = "".join(sentence)
    preprocessed_reviews.append(sentence)
"""

100%|████████████████████████████████████████████████████████████████████████████| 5053/5053 [00:02<00:00, 2447.32it/s]


In [60]:
from tqdm import tqdm

preprocessed_reviews = []

# Loop through each review and preprocess it
for sentence in tqdm(df2['Reviews'].values):
    # Remove non-alphabet characters and extra spaces using regex
    sentence = re.sub(r'[^a-zA-Z]', ' ', sentence)
    
    # Tokenize the sentence, convert to lowercase, and remove stopwords
    words = sentence.split()
    words = [word.lower() for word in words if word.lower() not in stopwords.words('english')]
    
    # Join the words back into a sentence with spaces
    preprocessed_sentence = ' '.join(words)
    
    # Append the preprocessed sentence to the list
    preprocessed_reviews.append(preprocessed_sentence.strip())

# Display the first few preprocessed reviews
print(preprocessed_reviews[:3])

100%|█████████████████████████████████████████████████████████████████████████████| 5053/5053 [00:20<00:00, 242.42it/s]

['hh', '', 'hotstar helps paymnt premium subscription transaction failed twice received refund one transaction']


In [61]:
preprocessed_reviews

['hh',
 '',
 'hotstar helps paymnt premium subscription transaction failed twice received refund one transaction',
 'hotstartweets currently jio network would like know whether able watch epl telecasted star sports select hd',
 'hotstartweets episodes sarabhai vs sarabhai season downloadable able watch offline please smthng',
 'hotstartweets able watch latest episode got app allow take screenshot error help resolve asap',
 'please allow rupay maestro payment gateways premium membership mean paytm works thru debit cards would great hotstartweets',
 'hotstar helps today epi lovekahaiintezaar nt available available morning showing nt available due expiry',
 'hotstartweets hotstarfraud paid subscription july havent received cashback specified hdfc card',
 'hotstartweets premium accnt hotstar showing tht premium member u pls chk ankush gmail com',
 'hotstartweets seeing blank page terms amp conditions hdfc bank cashback offer hotstar premium membership please help',
 'hotstartweets sir plea

## Feature Extraction

# TF-IDF:

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
x =tfidf.fit_transform(preprocessed_reviews).toarray()
pd.DataFrame(x).shape

(5053, 7142)

In [63]:
pd.DataFrame(x).head()

,0,1,2,3,4,5,6,7,8,9,...,7132,7133,7134,7135,7136,7137,7138,7139,7140,7141
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
df2['Sentiment_Polarity'] = df2['Sentiment_Polarity'].astype('category')
df2['Sentiment_Polarity'] = df2['Sentiment_Polarity'].cat.codes

In [65]:
df2['Sentiment_Polarity'].value_counts()

2    2513
1    1579
0     961
Name: Sentiment_Polarity, dtype: int64

In [66]:
# split the data into train and test

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,df2['Sentiment_Polarity'],test_size =0.25,random_state = 1)

## Random Forest Classifier

In [107]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()  
rf.fit(x_train,y_train)

RandomForestClassifier()

In [108]:
y_pred_rf_xtrain = rf.predict(x_train)
y_pred_rf_xtest = rf.predict(x_test)

In [109]:
print("Training Accuracy:",accuracy_score(y_train,y_pred_rf_xtrain))
print("*******************"*3)
print("Test Accuracy:",accuracy_score(y_test,y_pred_rf_xtest))

Training Accuracy: 0.9957772499340195
*********************************************************
Test Accuracy: 0.6890822784810127


## Cross Validation

In [110]:
from sklearn.model_selection import cross_val_score

training_accuracy = cross_val_score(rf,x_train,y_train)
training_accuracy.mean()

test_accuracy = cross_val_score(rf,x_test,y_test)
test_accuracy.mean()

0.6329506242549721

## Bernouli NB

In [73]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(x_train,y_train)

BernoulliNB()

In [75]:
y_pred_bnb_xtrain = bnb.predict(x_train)
y_pred_bnb_xtest = bnb.predict(x_test)

In [76]:
print("Training Accuracy:",accuracy_score(y_train,y_pred_bnb_xtrain))
print("*******************"*3)
print("Test Accuracy:",accuracy_score(y_test,y_pred_bnb_xtest))

Training Accuracy: 0.7326471364476115
*********************************************************
Test Accuracy: 0.5791139240506329


In [77]:
print(confusion_matrix(y_train,y_pred_bnb_xtrain))
print("***************"*6)
print(confusion_matrix(y_test,y_pred_bnb_xtest))

[[ 335   74  315]
 [  13  572  579]
 [   8   24 1869]]
******************************************************************************************
[[ 50  59 128]
 [ 27 118 270]
 [ 10  38 564]]


In [79]:
print(classification_report(y_train,y_pred_bnb_xtrain))
print("***************"*6)
print(classification_report(y_test,y_pred_bnb_xtest))

              precision    recall  f1-score   support

           0       0.94      0.46      0.62       724
           1       0.85      0.49      0.62      1164
           2       0.68      0.98      0.80      1901

    accuracy                           0.73      3789
   macro avg       0.82      0.65      0.68      3789
weighted avg       0.78      0.73      0.71      3789

******************************************************************************************
              precision    recall  f1-score   support

           0       0.57      0.21      0.31       237
           1       0.55      0.28      0.37       415
           2       0.59      0.92      0.72       612

    accuracy                           0.58      1264
   macro avg       0.57      0.47      0.47      1264
weighted avg       0.57      0.58      0.53      1264



In [82]:
training_accuracy = cross_val_score(bnb,x_train,y_train)
print(training_accuracy.mean())

test_accuracy = cross_val_score(bnb,x_test,y_test)
print(test_accuracy.mean())

0.5734990571726332
0.49762532153836503


## GaussNB

In [85]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(x_train,y_train)

GaussianNB()

In [86]:
y_pred_gnb_train = gnb.predict(x_train)
y_pred_gnb_test = gnb.predict(x_test)

In [87]:
print(confusion_matrix(y_train,y_pred_gnb_train))
print()
print(confusion_matrix(y_test,y_pred_gnb_test))

[[ 724    0    0]
 [ 223  941    0]
 [ 529  212 1160]]

[[149  38  50]
 [198 142  75]
 [258 151 203]]


In [88]:
print(classification_report(y_train,y_pred_gnb_train))
print()
print(classification_report(y_test,y_pred_gnb_test))

              precision    recall  f1-score   support

           0       0.49      1.00      0.66       724
           1       0.82      0.81      0.81      1164
           2       1.00      0.61      0.76      1901

    accuracy                           0.75      3789
   macro avg       0.77      0.81      0.74      3789
weighted avg       0.85      0.75      0.76      3789


              precision    recall  f1-score   support

           0       0.25      0.63      0.35       237
           1       0.43      0.34      0.38       415
           2       0.62      0.33      0.43       612

    accuracy                           0.39      1264
   macro avg       0.43      0.43      0.39      1264
weighted avg       0.49      0.39      0.40      1264



In [89]:
print(accuracy_score(y_train,y_pred_gnb_train))
print()
print(accuracy_score(y_test,y_pred_gnb_test))

0.7455793085246767

0.39082278481012656


In [90]:
training_accuracy = cross_val_score(gnb,x_train,y_train)
print(training_accuracy.mean())

test_accuracy = cross_val_score(gnb,x_test,y_test)
print(test_accuracy.mean())

0.40010351930791943
0.476275174101261
